PREPROCESSING

In [1]:
import pandas as pd
import numpy as np

fields=['UMUR','JENIS_KELAMIN','KECAMATAN' ,'FOTO_TORAKS','STATUS_HIV',
'RIWAYAT_DIABETES','LOKASI_ANATOMI']
dataset=pd.read_csv('toy15.csv',usecols=fields,delimiter=";")

#definisi atribut
umur=dataset.UMUR.values
jenis_kelamin=dataset.JENIS_KELAMIN.values
kecamatan=dataset.KECAMATAN.values
foto_toraks=dataset.FOTO_TORAKS.values
status_hiv=dataset.STATUS_HIV.values
riwayat_diabetes=dataset.RIWAYAT_DIABETES.values
target=dataset.LOKASI_ANATOMI.values

jumlahdata=len(dataset)
umur_max = max(umur)
umur_min = min(umur)
datanew=[]
for i in range(jumlahdata):
    labelumur= 0 if(umur[i]<=5) else 1 if(umur[i]>= 6 and umur[i]<=11) else 2 if(umur[i]>= 12 and umur[i]<=16) else 3 if(umur[i]>= 17 and umur[i]<=25) else 4 if(umur[i]>= 26 and umur[i]<=35) else 5 if(umur[i]>= 36 and umur[i]<=45) else 6 if(umur[i]>= 46 and umur[i]<=55) else 7 if(umur[i]>= 56 and umur[i]<=65) else 8 if(umur[i]>=66) else None
    labeljk= 1 if(jenis_kelamin[i]=="L") else 0 
    labelkec=kecamatan[i]
    labelft= 1 if(foto_toraks[i]=="Positif") else 0 if(foto_toraks[i]=="Negatif") else None
    labelsh= 1 if(status_hiv[i]=="Positif") else 0 if(status_hiv[i]=="Negatif") else None
    labelrd= 1 if(riwayat_diabetes[i]=="Ya") else 0 if(riwayat_diabetes[i]=="Tidak") else None
    # labeltcm= 2 if(hasil_tcm[i]=="Rif Resisten") else 1 if(hasil_tcm[i]=="Rif Sensitif") else 0 if(hasil_tcm[i]=="Negatif") else None
    labeltarget= 0 if(target[i]=="Paru") else 1 if(target[i]=="Ekstra Paru") else 1
    datanew.append([labelumur,labeljk,labelkec,labelft,labelsh,labelrd,labeltarget])
udtf= pd.DataFrame(datanew,columns=fields)
# print(udtf)
udtf.to_csv('D:/KULIAH/SKRIPSI/SIDANG/normalisasi_toy15.csv')

MISSING VALUE KNN

In [2]:
from math import sqrt
from re import L
import pandas as pd
import numpy as np
import statistics
from statistics import mode

fields=['UMUR','JENIS_KELAMIN','KECAMATAN', 'FOTO_TORAKS','STATUS_HIV',
'RIWAYAT_DIABETES','LOKASI_ANATOMI']
datasetNorm=pd.read_csv('normalisasi_toy15.csv',usecols=fields)

#definisi atribut
umur=datasetNorm.UMUR.values
jenis_kelamin=datasetNorm.JENIS_KELAMIN.values
kecamatan=datasetNorm.KECAMATAN.values
foto_toraks=datasetNorm.FOTO_TORAKS.values
status_hiv=datasetNorm.STATUS_HIV.values
riwayat_diabetes=datasetNorm.RIWAYAT_DIABETES.values
lokasi_anatomi=datasetNorm.LOKASI_ANATOMI.values

In [46]:
banyak_data = len(datasetNorm)
print("JUMLAH DATA REAL:",banyak_data)
sum_mv = datasetNorm.isna().sum() 
print("======MISSING VALUE======\n",sum_mv)

k=9 
missingData, fullData=[],[] #menyimpan index dari record data kosong & index dari record data penuh
missKol_List=[]
e_dis=[]
indexSortAll,neighborAll=[],[]

def getData(row):
    rowData = []
    getDataRow = datasetNorm.iloc[row].values
    for i in getDataRow:
        rowData.append(i)
    return rowData

def getFullData(fullData):
    for i in range(banyak_data):
        if(pd.notnull(getData(i)[0]) and pd.notnull(getData(i)[1]) and pd.notnull(getData(i)[2])
        and pd.notnull(getData(i)[3]) and pd.notnull(getData(i)[4]) and pd.notnull(getData(i)[5]) 
        and pd.notnull(getData(i)[6])):
            fullData.append(i)
        else:
            None
    print("FUll",fullData)
    fullFrame= pd.DataFrame(getData(fullData),columns=fields)
    # print(fullFrame)
    fullFrame.to_csv('D:/KULIAH/SKRIPSI/SIDANG/fullFrame_toy15.csv')
    print("\nJUMLAH DATA RECORD NOT MISSING:",len(fullData))

def getMissingData(missingData):
    for i in range(banyak_data):
        if(i not in fullData):
            missingData.append(i)
    # print("MISSING",missingData)
    missingFrame= pd.DataFrame(getData(missingData),columns=fields)
    # print(missingFrame)
    missingFrame.to_csv('D:/KULIAH/SKRIPSI/SIDANG/missingFrame_toy15.csv')
    print("JUMLAH DATA RECORD MISSING:",len(missingData))
    for i in missingData:
        for j in range(len(getData(i))):
            if(pd.isnull(getData(i)[j])):
                missKol_List.append([i,j])
    # print("MISSING IN COLOUMN",missKol_List,"\n")
    print("JUMLAH SELURUH FITUR / ATRIBUT MISSING:",len(missKol_List))

def euclideanStep():
    #EUCLIDEAN
    for i in range(len(missKol_List)):
        missingLine=missKol_List[i][0]
        dis_kol =[]
        for j in range(len(fullData)):
            fullLine=fullData[j]
            # print(i,"missing point:",getData(missingLine),">>>other point:",getData(fullLine))
            numRow = len(getData(missingLine)) #panjang fitur/panjang atribut
            euc=0
            # for x in range(numRow):
            for x in range(numRow-1):
                if(type(getData(missingLine)[x]) != str and type(getData(fullLine)[x]) != str and pd.notnull(getData(missingLine)[x]) and pd.notnull(getData(fullLine)[x])):
                    euc+=(getData(missingLine)[x] - getData(fullLine)[x]) * (getData(missingLine)[x] - getData(fullLine)[x])
            eucValue=sqrt(euc)
            dis_kol.append(eucValue)
        e_dis.append(dis_kol)
        indexSort=np.argsort(e_dis[i])[:k]
        indexSortAll.append(indexSort)
    # print("edis",e_dis)
    print("indexSortAll",indexSortAll)
    euclideanToCsv()
    

def euclideanToCsv():
    field=[]
    for x in range(len(e_dis[0])):
        field.append(x)

    print(field)
    euclideanToy15_knn= pd.DataFrame(e_dis,columns=field)
    print(euclideanToy15_knn)
    euclideanToy15_knn.to_csv('D:/KULIAH/SKRIPSI/SIDANG/euclideanToy15_knn.csv')
    
def neighborStep():
    for x in range(len(indexSortAll)):
        neighbor=[]
        for y in range(len(indexSortAll[x])):
            indexNeighbor=indexSortAll[x][y]
            neighbor.append(fullData[indexNeighbor])
        neighborAll.append(neighbor)
    # print(neighborAll,"\n")

toReplace=[]
def mostFrequentStep():
    for x in range(len(missKol_List)):
        neighborLabel=[]
        for y in range(len(neighborAll[x])):
            missingCol=missKol_List[x][1]
            neighborCol=neighborAll[x][y]
            neighborValue=getData(neighborCol)[missingCol]
            # print(x,"KOL",missingCol,"neighbor",neighborCol,">>>value",neighborValue)
            neighborLabel.append(neighborValue)
        mostFrequent=mode(neighborLabel)
        # print("missing index baris",missKol_List[x][0],"index kolom",missingCol,">>>most frequent",mostFrequent)
        toReplace.append([missKol_List[x][0],missingCol,mostFrequent])
    # print(toReplace)

changed_data = []
def imputationStep():
    for i in range(banyak_data):
        baris = getData(i)
        # print(i,baris)
        for j in range(len(baris)):
            for x in range(len(toReplace)):
                if(i == toReplace[x][0] and j == toReplace[x][1]):
                    baris[j]=toReplace[x][2]
        # print(i,'>>>changed',baris)
        changed_data.append(baris)
    # print('clean data :',changed_data)

def toDataFrame():
    updateToy15_knn= pd.DataFrame(changed_data,columns=fields)
    print(updateToy15_knn)
    updateToy15_knn.to_csv('D:/KULIAH/SKRIPSI/SIDANG/updateToy15_knn.csv')


full=getFullData(fullData)
missing=getMissingData(missingData)
euc=euclideanStep()
neighbor=neighborStep()
mostFrequent=mostFrequentStep()
imputation=imputationStep()
dframe=toDataFrame()

   

JUMLAH DATA REAL: 15
======MISSING VALUE======
 UMUR                0
JENIS_KELAMIN       0
KECAMATAN           0
FOTO_TORAKS         2
STATUS_HIV          2
RIWAYAT_DIABETES    1
LOKASI_ANATOMI      0
dtype: int64
FUll [0, 1, 4, 6, 7, 8, 9, 11, 12, 13, 14]

JUMLAH DATA RECORD NOT MISSING: 11
JUMLAH DATA RECORD MISSING: 4
JUMLAH SELURUH FITUR / ATRIBUT MISSING: 5
indexSortAll [array([ 5,  8,  6,  1,  7, 10,  2,  3,  9], dtype=int64), array([ 7, 10,  0,  1,  2,  3,  4,  9,  6], dtype=int64), array([ 7, 10,  0,  1,  2,  3,  4,  9,  6], dtype=int64), array([ 0,  4,  7, 10,  1,  2,  3,  9,  6], dtype=int64), array([ 7, 10,  1,  2,  3,  6,  9,  0,  4], dtype=int64)]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        0         1         2         3        4         5         6   \
0  5.09902  4.123106  4.242641  4.242641  5.09902  1.414214  2.449490   
1  1.00000  1.000000  1.000000  1.000000  1.00000  5.000000  2.000000   
2  1.00000  1.000000  1.000000  1.000000  1.00000  5.000000  2.000000   
3  